In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
data_path = '/content/gdrive/MyDrive/Evaluations.csv'

In [ ]:
!pip install nltk


In [ ]:
!pip install bert_score

In [ ]:
import numpy as np
import re

def remove_source(text):
    cleanText = text
    if '(Source' in cleanText:
        cleanText,_,_ = cleanText.partition('(Source')
    elif '[Written ' in cleanText:
        cleanText,_,_ = cleanText.partition('[Written')

    return cleanText

def clean_synopsis(data):
    synopsis = data['Synopsis']
    synopsis = synopsis.apply(lambda x: str(x))

    # removing very small synopsis
    synopsis = synopsis.apply(lambda x: x if ((len(str(x).strip().split())<=300) and len(str(x).strip().split())>30  ) else -1)
    synopsis = synopsis[synopsis!=-1]

    # removing source text
    synopsis = synopsis.apply(lambda x: remove_source(x))

    # removing japanese characters
    synopsis = synopsis.apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))

    # remove symbols
    rx = re.compile('[&#/@`)(;<=\'"$%>]')
    synopsis = synopsis.apply(lambda x: rx.sub('',x))
    synopsis = synopsis.apply(lambda x: x.replace('>',""))
    synopsis = synopsis.apply(lambda x: x.replace('`',""))
    synopsis = synopsis.apply(lambda x: x.replace(')',""))
    synopsis = synopsis.apply(lambda x: x.replace('(',""))


    return synopsis.reset_index(drop=True)

In [ ]:
import pandas as pd
# Load the dataset
dataset_path = '/content/gdrive/MyDrive/anime_info.csv'
df_Data = pd.read_csv(dataset_path)

In [ ]:
cleaned_synopsis = clean_synopsis(df_Data)
cleaned_synopsis_list = list(cleaned_synopsis)
reference_text = ' '.join(cleaned_synopsis_list)

In [ ]:
!pip install --upgrade jax jaxlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 9.0 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.4.14-py3-none-any.whl size=1535364 sha256=c16d01f229bb562f9e39b0c985237fb3579cad96d053d4a3d7aa3d8353b7cd3f
  Stored in directory: /root/.cache/pip/wheels/85/52/e7/dfa571c9f9b879e3facaa1584f52be04c4c3d1e14054ef40ab
Successfully built jax
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.13+cuda11.cudnn86
    Uninstalling jaxlib-0.4.13+cuda11.cudnn86:
      Successfully uninstalled jaxlib-0.4.13+cuda11.cudnn86
  Attempting uninstall: jax
    Found existing installation: jax 0.4.13
    Uninstalling jax-0.4.13:
      Successfully uninstalled jax-0.4.13


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from itertools import combinations
from bert_score import score

nltk.download('punkt')

#self-BLEU
def compute_self_bleu(sentences):
    self_bleu_scores = []

    for i, j in combinations(range(len(sentences)), 2):
        reference_1 = sentences[i].split()
        reference_2 = sentences[j].split()

        self_bleu = sentence_bleu([reference_1], reference_2, smoothing_function=SmoothingFunction().method7)
        self_bleu_scores.append(self_bleu)

    if not self_bleu_scores:  # Check if the list is empty
        return 1  # Return a default value if no scores were calculated

    average_self_bleu = sum(self_bleu_scores) / len(self_bleu_scores)

    return average_self_bleu

#BERTScore
def compute_bert_score(story):
    P, R, F1 = score([story], [reference_text], lang='en', verbose=False)
    bert_score = F1.mean().item()
    return bert_score

df = pd.read_csv(data_path)

# Calculate self-BLEU for each story
df['self-bleu'] = df['Story'].apply(compute_self_bleu)

# Calculate BERTScore for each story
df['bert-score'] = df['Story'].apply(compute_bert_score)


df.to_csv("final_scores_with_self_bleu_and_bert_score.csv", index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [ ]:
df

,Story,Story Tag,Average Human Score,self-bleu,bert-score
0,"In an alternative universe, the human race has...",GPT2_FINE_TUNED,6.28125,0.008541,0.818203
1,"In an alternative universe, the Earth has been...",GPT2_FINE_TUNED,6.28125,0.008592,0.815351
2,In an alternative universe set in the distant ...,GPT3_FINE_TUNED,7.37500,0.008497,0.818651
3,In an alternative universe there existed a wor...,GPT3_FINE_TUNED,8.06250,0.008037,0.804716
4,"In an alternative universe, the universe where...",GPT2_ZERO,3.68750,0.010791,0.788979
5,"One day, a small girl named Natsume found a bo...",GPT3_ZERO,5.78125,0.007966,0.802239
6,"In an alternative universe, humans coexisting ...",GPT4_ZERO,7.03215,0.008271,0.814587


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

df['self-bleu'] = 1 + 9 * (1 - df['self-bleu'])  # Lower self-bleu is better, so we subtract it from 1
df['bert-score'] = 1 + 9 * df['bert-score']  # Assuming higher bert-score is better


file_path = '/content/gdrive/My Drive/final_scores_with_transformed_self_bleu_and_bert_score.csv'


df.to_csv(file_path, index=False)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
original_scores = [0.008541, 0.008592, 0.008497, 0.008037, 0.010791, 0.007966, 0.008271]

# Transform the scores
transformed_scores = [1 + 9 * (1 - score) for score in original_scores]

print(transformed_scores)


[9.923131, 9.922672, 9.923527, 9.927667, 9.902881, 9.928306, 9.925561]
